<a href="https://colab.research.google.com/github/wdempsey/AI4Health-Online-Experimentation/blob/main/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online learning and experimentation algorithms in mobile health

In [30]:
## Import necessary 
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression

#Part 1: Overview on Contextual Bandits

- For each person in a study, let $t=1,\ldots, T$ denote a sequence of decision points.  
- At each decision time $t$,  we observe a state variable $S_t \in \mathbb{R}^p$.  
- After observing the state variable $S_t$, the _agent_ decides to take action $A_t \in \mathcal{A}$.  
- After observing state $S_t$ and taking action $A_t$, the agent receive a reward $R_t$ given by
$$
R_t = r(S_t, A_t) + \epsilon_t
$$
where $r(c,a)$ is a function that maps the state-action pair onto the real line and $\epsilon_t$ is a random error term, e.g., $\mathbb{E} [\epsilon_t] = 0$. 
- The triple (context, action, reward) at a sequence of decision points defines a _contextual bandit_ setting.  
- Here, the goal is to maximize the expected reward at every time point $\mathbb{E}[R_t \mid S_t, A_t=a] = r(S_t, a)$. 
- If we knew the reward function $r: \mathcal{S} \times \mathcal{S} \to \mathbb{R}$, then the optimal action given state $s$ is
$$
a^\star (s) = \max_{a \in \mathcal{A}} r(s, a)
$$

### A simple approach:

- Consider $\mathcal{A} =\{0,1\}$
- Randomize treatment $A_t \sim \text{Bern}(p)$ for $t=1,\ldots,T$
- Then for $t>T$, just choose
$$
\hat A^\star_t = \max_{a \in A} \hat r(S_t, a)
$$
where $\hat r(s,a)$ is the model fit using the batch data collected.

### Linear Contextual Bandit

- Assume that the reward structure follows
$$
r(s,a) = x(s,a)^\top \beta 
$$
where $x(s,a) \in \mathbb{R}^{p}$ is a $p$-dimensional summary of the state and $\beta \in \mathbb{R}^p$ is an unknown parameter.

In [43]:
# Simulation example
T = 200 # number of steps

## Generate context (normal and binary states)
mu, sigma = 0, 1 # mean and standard deviation
state1 = np.random.normal(mu, sigma, T) # Continuous state
state2 = np.random.binomial(n=1, p = 0.7,size=T) # Binary state
state = np.stack((state1,state2), axis = 1) # Compelte State at each time

## Generate actions (MRT with probability  )
action = np.random.binomial(n=1, p = 0.5,size=T) # Binary state

## Generate true reward
def reward(state, action):
  base_reward = state[0] + 0.3*state[1] 
  advantage = 0.5*state[0] - 0.7*state[1]
  return base_reward + advantage * action

y = np.repeat(0.,T)
for t in range(T):
  y[t] = reward(state[t,:], action[t]) + np.random.normal(0, 1, 1)


## Triple
triple = np.column_stack((state,action, y))
print("First 10 entries of state (2D), action, and reward")
print(triple[1:10,:])
print("\n")

## Build the design matrix
X = state
for col in range(2):
  temp = np.multiply(state[:,col],action)
  X = np.column_stack((X, temp))

reg = LinearRegression().fit(X,y)
print("True coefficients using linear model")
print(np.array([1,0.3,0.5,-0.7]))
print("Fitted coefficients using linear model")
print(reg.coef_)





First 10 entries of state (2D), action, and reward
[[ 0.4889732   0.          0.          0.0970272 ]
 [-1.65526392  1.          0.         -2.15776888]
 [ 0.88552154  0.          0.          1.29955221]
 [-1.21393632  1.          0.          0.97508805]
 [-1.79251348  1.          0.         -1.62145004]
 [ 1.42910115  0.          1.          1.68040017]
 [-2.05180643  1.          0.         -3.29626183]
 [ 0.2045409   1.          0.          1.70142421]
 [ 2.92629192  1.          0.          1.87006771]]


True coefficients using linear model
[ 1.   0.3  0.5 -0.7]
Fitted coefficients using linear model
[ 1.08134331  0.49970343  0.50450428 -0.64743794]


## Question 1: What aspects of the reward impact decision making?

- The only thing that impacts decision to choose $a = 1$ or $a=0$ is the _advantage function_:
$$
A(s) = r(s,1) - r(s,0)
$$
- In the example above
$$
A(s) = 0.5 s_{0} - 0.7 s_{1} > 0 \Rightarrow \frac{0.5}{0.7} s_0 > s_1
$$




## Question 2: What are the pros of this simple approach?  What are the cons?  

- Why may we not want to use a randomized policy to collect data in mobile health?


Pros (non-exhaustive)
- Simple algorithm
- With sufficient data will construct a 'good' policy
- Easy to explain 

Cons (non-exhaustive)
- Exploration is random so we learn slowly about the space
- Exploit policy may not be optimal
- How do we know that we collected enough data? 

## Regret and bandit algorithms

- To the con

- What are some reasons 

# Part 1b: Investigating mHealth randomized trial data

In HeartSteps V2, decision points are 6 times per day.  
Variables include
- __ID (string)__ is a unique identifier for the decision
Heartsteps ID (string) is the ID for the participant in the study
Test (boolean) is if the walking suggestion was a test explicitly sent to the participant, and shouldn’t be included in analysis
- __Decision time (datetime string YYYY-MM-DD HH:MM:SS)__ is the time the decision to treat or not treat the participant was made. This time might be slightly different than the time the participant was sent or received the walking suggestion push notification. The time of day is reported in a 24 hour format and is localized to the participant’s timezone at the decision time.
Sedentary (boolean) if the participant is sedentary, it means HeartSteps had recorded less than 250 steps in the last 60 minutes.
- __Treated (boolean)__ indicates that the participant was randomized to receive a walking suggestion at the time of the walking suggestion decision. This value is typically generated by the walking-suggestion-service.
- __Treatment Probability (decimal between 0 and 1)__ this is the probability that the participant would be sent a walking suggestion. 1 means the participant will be sent a walking suggestion, 0 means the participant will not be sent a walking suggestion. If Available is false, then the treatment probability should be zero. This value is typically generated by the walking-suggestion-service.
- __Notification Title (string)__ is the title of the push notification sent to the participant. If this field is missing, then a walking suggestion wasn’t sent to the participant.
- __Notification Message (string)__ is the message that was sent to the participant. If this field is missing, then a walking suggestion wasn’t sent.
Sent Time (datetime string YYYY-MM-DD HH:MM:SS) is the time that the walking suggestion decision was sent to the participant, which can be slightly different than the time the participant received the message. Hours are reported using a 24 hour clock, the timezone for this message is the participant’s current timezone. If this value is missing, it means the participant was randomized to not receive a message, or the decision was imputed (see below).
- __Engaged Time (datetime string YYYY-MM-DD HH:MM:SS)__ is when the participant clicked “Ok” at the bottom of the walking suggestion in the heartsteps-app. Hours are reported using a 24 hour clock, the timezone for this message is the participant’s current timezone. This value will be missing if the participant opened the push notification, but closed the heartsteps-app before clicking “Ok” -- this value could also be missing for the same reasons as sent time.
- __Location (string)__ is the participant’s location represented as a category at the walking suggestion decision time. The category is determined by comparing the participant’s last GPS location reported by the heartsteps-clock-face, to the list of places the participant entered during the onboarding process -- if the participant is within 500 meters of a defined place, they are determined to be at that place. Possible values are “home” “work” or “other”. It’s possible for this field to be empty, which indicates that we haven’t received a location record from the participant within the last 60 minutes.
- __Temperature (integer)__ the temperature in fahrenheit for the participant’s location at the walking suggestion decision time as reported by the DarkSky API. If there is no reported location for the walking suggestion decision, then the temperature reported is the average temperature for each place the participant defined in the onboarding process.
- __Precipitation Type (string)__ is the type of precipitation reported by the DarkSky API for the participant’s current location. Possible values are “None” “Rain” and “Snow”. If the participant’s location is missing at the walking suggestion time, this value represents the most extreme value from each place the participant defined during onboarding (eg Snow is more extreme than Rain, which is more extreme than None).
- __Precipitation Probability (decimal between 0 and 1)__ is the probability of precipitation reported by the DarkSky API for the participant’s current location. If the participant’s location is missing, then this value is the average precipitation probability at each place that a participant defined during onboarding.



In [21]:
## Reading in the data


## ADD CODE FOR EXPLORATORY DATA ANALYSIS HERE


## Part 1c: Bandits in mHealth

## Part 1d: Going beyond Bandits



```
# This is formatted as code
```



# Part 2: Constrained Optimization